# Data Analysis Extra - Playlists

Como análisis extra pensé que podría analizar las plasyslist que crearon lso usuarios durante la pandemia, podría dar indicadores de qué tipo de música prefiere escuchar durante una pandemia.

el proceso a realizar es buscar las playlist que tengan en el nombre alguna palabra relacionada con la pandemia, me he limitado al español

In [95]:
# Ver : https://towardsdatascience.com/what-music-do-people-listen-to-during-the-coronavirus-a-report-using-data-science-1a2035d12430
# Leer: https://blog.chartmetric.com/covid-19-effect-on-the-global-music-business-part-1-genre/
# https://gist.github.com/ilias1111/e503bbab0a98c20377686cc75ffad451

In [96]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from datetime import datetime

# Meter credenciales de Spotify
passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Vemos como está montada la estructura de información en la API de Spotify, para buscar una palabra en los nombres de las playlists
print(sp.search('q=fernando simón', type='playlist'))
# ["covid","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=q%3Dfernando+sim%C3%B3n&type=playlist&offset=0&limit=10', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.co

In [61]:
# Creamos un script ya completo para buscar las palabras y sacar solo el total de listas que contiene esta palabra en alguna parte de su información
for word in ["fernando simón"]:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['items'])
    
    

[{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1/tracks', 'total': 271}, 'type': 'playlist', 'uri': 'spotify:playlist:01bawF2DM0YzefGbC54Ua1'}

In [68]:
# Extracción de las playlist con dichos nombres, domcumentación: https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/
# "added_at":i["tracks"]['items'][0]["added_at"]
# Función para extrar determinados features de la playlist
# ['items']["tracks"]['items'][0]["added_at"]
def batch_proccess(x,lista):
    for i in x['playlists']['items']:
        lista.append({"name" : i['name'], "total":i["tracks"]['total'], 'id':i['id'], "uri":i["uri"], "URL":i["tracks"]['href']})

# Sacar un listado de las playlist, en bloques de 50, que es el máximo que te permite la API
list_of_playlists = []
for term in ["covid","cuarentena",'pandemia','covid19', "coronavirus", 'fernando simón']:
    for i in range(0,2000, 50):
        try:
            init_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
            batch_proccess(init_data,list_of_playlists)
        except:
            print("Error")

In [97]:
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)
playlists

,name,total,id,uri,URL
0,Calma,151,37i9dQZF1DWY5LGZYBBHHz,spotify:playlist:37i9dQZF1DWY5LGZYBBHHz,https://api.spotify.com/v1/playlists/37i9dQZF1...
1,quarantine Vibes | Covid 19 |,40,6zjC569rXoYGOLwPUt37WW,spotify:playlist:6zjC569rXoYGOLwPUt37WW,https://api.spotify.com/v1/playlists/6zjC569rX...
2,COVID-19 Quarantine Party,29,1Tn6OrkJhOYO5u6y16erqB,spotify:playlist:1Tn6OrkJhOYO5u6y16erqB,https://api.spotify.com/v1/playlists/1Tn6OrkJh...
3,Canciones Mas Virales que el Coronavirus 🦠🎶,100,4Y2h9TZNZersxSYeXujCNu,spotify:playlist:4Y2h9TZNZersxSYeXujCNu,https://api.spotify.com/v1/playlists/4Y2h9TZNZ...
4,Drunksouls COVID19 quarantine discovery: music...,49,64KUBwKGkUEqIU08HDhJ94,spotify:playlist:64KUBwKGkUEqIU08HDhJ94,https://api.spotify.com/v1/playlists/64KUBwKGk...
...,...,...,...,...,...
5577,My Playlist 3.5,42,2FykDxR75paDjs4cYDWAZX,spotify:playlist:2FykDxR75paDjs4cYDWAZX,https://api.spotify.com/v1/playlists/2FykDxR75...
5578,nCov-2019,43,3cqPni9PbXLE7WukiRN9Rc,spotify:playlist:3cqPni9PbXLE7WukiRN9Rc,https://api.spotify.com/v1/playlists/3cqPni9Pb...
5579,FS Corona-voidance Jams,13,4AntDyOlereHExywYCgZ1j,spotify:playlist:4AntDyOlereHExywYCgZ1j,https://api.spotify.com/v1/playlists/4AntDyOle...
5580,Isolamento com Ashton,11,2L41j2odRvSvBZnH4AZcdV,spotify:playlist:2L41j2odRvSvBZnH4AZcdV,https://api.spotify.com/v1/playlists/2L41j2odR...


In [ ]:
'''# Loop para sacar features de todas las canciones de una lista
# OJO, tarda bastante dependendo del número de canciones, por lo que antes hago un testeo
import time
list_toextract = test_list

tracks = []

Time1 = datetime.now()


for i in range(len(list_toextract)):
    time.sleep(.5)
    track = getTrackFeatures(list_toextract[i])
    tracks.append(track)
    
Time2 = datetime.now()


Time1_v = datetime.now()

for i in range(len(list_toextract)):
    # time.sleep(.5)
    track = getTrackFeatures(list_toextract[i])
    tracks.append(track)
    
Time2_v = datetime.now()

print("Tiempo 1:", Time2 -Time1)
print("Tiempo 2:", Time2_v -Time1_v)
'''

In [70]:
# Sacar las canciones de las listas
songs = []
for uri in playlists['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    songs.append(k['track']["uri"])
            except:
                pass

In [98]:
len(songs)

635501

In [100]:
from collections import Counter
data_playlist_songs = pd.DataFrame.from_dict(Counter(songs), orient='index').reset_index().rename(columns={"index":"uri", 0:"count"})
data_playlist_songs.sort_values(by=['count'], ascending=False)

,uri,count
735,spotify:track:2DEZmgHKAvm41k4J3R2E9Y,563
315,spotify:track:0SqqAgdovOE24BzxIClpjw,514
759,spotify:track:7k4t7uLgtOxPwTpFmtJNTY,420
193,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,373
2241,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,373
...,...,...
117517,spotify:track:0ZdSPHnoFUAE1tck8oxSyJ,1
117518,spotify:track:3iua7Qv6SauKb6AIRErIDE,1
117519,spotify:track:2yTTVFwrD7a4liRK2bvflX,1
117520,spotify:track:2BsUnm9Hsy92ESAeOtzXtl,1


In [105]:
# Ver cuántas canciones hay en el dataframe que se repiten más de 9 veces 
data_playlist_songs[(data_playlist_songs['count']>9) & (data_playlist_songs['count']<900)].describe()

,count
count,9187.000000
mean,25.166322
std,29.246190
min,10.000000
25%,12.000000
50%,16.000000
75%,26.000000
max,563.000000


In [108]:
def data_enrich(x):
    
    data = sp.track(x['uri'])

    name = data['name']
    pop = data['popularity']
    date = data['album']['release_date']

    features = sp.audio_features(x['uri'])



    return {**x, "name" : name, 'release_date': date, "popularity" : pop,
            'explicit' : data['explicit'] , **features[0] }

In [ ]:
data_playlist_songs_final = data_playlist_songs.apply(lambda x : data_enrich(x) , axis=1, result_type="expand")

In [ ]:
data_playlist_songs_final = data_playlist_songs_final.drop(columns=['id','track_href','analysis_url','time_signature','type'])
data_playlist_songs_final 